In [1]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Asheville_airbnb import Asheville_airbnb
import datetime as dt
from config import api_key

importing Jupyter notebook from Asheville_airbnb.ipynb


Asheville_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
Asheville_airbnb.ipynb:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "cells": [
Asheville_airbnb.ipynb:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "metadata": {},


ModuleNotFoundError: No module named 'config'

In [3]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Asheville, NC")

ASH_coord = []
ASH_coord.append(location.latitude)
ASH_coord.append(location.longitude)

ASH_coord

[35.6009498, -82.5540161]

In [4]:
forecast = forecastio.load_forecast(api_key, ASH_coord[0], ASH_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Rain on Thursday and Friday, with high temperatures bottoming out at 42°F on Monday.
rain
56


In [5]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [6]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [7]:
ASH_table = pd.merge(df, Asheville_airbnb, on="Date")
ASH_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-22,43.64,27.87,0.21,0.0008,rain,75,323,223,110,61
1,2018-12-23,52.04,27.79,0.22,0.0015,rain,97,408,196,93,58
2,2018-12-24,41.76,25.00,0.16,0.0005,snow,90,369,198,88,55
3,2018-12-25,45.75,28.86,0.00,0.0000,none,92,361,197,88,57
4,2018-12-26,56.00,33.44,0.00,0.0000,none,77,363,187,79,52
5,2018-12-27,46.92,47.31,0.83,0.0230,rain,77,347,153,58,47
6,2018-12-28,66.41,43.96,0.71,0.0128,rain,57,264,184,76,41
7,2018-12-29,59.16,41.69,0.00,0.0000,none,55,246,180,76,40
